In [114]:
import ROOT
import numpy as np
import matplotlib.pyplot as plt
import sndisplay as sn
import uproot as uproot
import pandas as pd

In [115]:
from matplotlib import cycler, patches

IPython_default = plt.rcParams.copy()
SMALL_SIZE = 8
MEDIUM_SIZE = 10
BIGGER_SIZE = 12

figsize = (4.5, 3)

plt.rc('font', size=SMALL_SIZE)  # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)  # fontsize of the axes title
plt.rc('axes', labelsize=SMALL_SIZE)  # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)  # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)  # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)  # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
plt.rcParams["font.family"] = "serif"

marker_size = 3
line_width = 0.5

colors = cycler('color', ['#EE6666', '#3388BB', '#9988DD', '#EECC55', '#88BB44', '#FFBBBB'])
plt.rc('axes', facecolor='white', edgecolor='black',
       axisbelow=True, grid=True, prop_cycle=colors)

In [116]:
input_file_name = "/Users/williamquinn/Desktop/read_red/snemo_run-619_red.root"
file = ROOT.TFile(input_file_name, "READ")
file.ls()
tree = file.RED
tree.Print()

TFile**		/Users/williamquinn/Desktop/read_red/snemo_run-619_red.root	
 TFile*		/Users/williamquinn/Desktop/read_red/snemo_run-619_red.root	
  KEY: TTree	RED;1	SuperNEMO RED data
******************************************************************************
*Tree    :RED       : SuperNEMO RED data                                     *
*Entries :     2202 : Total =        35348919 bytes  File  Size =    6198511 *
*        :          : Tree compression factor =   5.71                       *
******************************************************************************
*Br    0 :run_id    : run_id/I                                               *
*Entries :     2202 : Total  Size=       9369 bytes  File Size  =        153 *
*Baskets :        1 : Basket Size=      32000 bytes  Compression=  58.04     *
*............................................................................*
*Br    1 :event_id  : event_id/I                                             *
*Entries :     2202 : Total  Siz

In [120]:
branches = tree.GetListOfBranches()
print([branch.GetName() for branch in branches])

['run_id', 'event_id', 'event_clock', 'event_ticks', 'nb_calo_hits', 'calo_hit_id', 'calo_om_side_id', 'calo_om_wall_id', 'calo_om_column_id', 'calo_om_row_id', 'calo_om_ref_id', 'calo_clock', 'calo_ticks', 'calo_lto', 'calo_ht', 'calo_fcr', 'calo_lt_trigger_counter', 'calo_lt_time_counter', 'calo_fwmeas_baseline', 'calo_fwmeas_peak_amplitude', 'calo_fwmeas_peak_cell', 'calo_fwmeas_charge', 'calo_fwmeas_rising_cell', 'calo_fwmeas_falling_cell', 'calo_waveform', 'nb_tracker_hits', 'tracker_hit_id', 'tracker_cell_side_id', 'tracker_cell_row_id', 'tracker_cell_layer_id', 'tracker_clock', 'tracker_anode_R0_ticks', 'tracker_anode_R1_ticks', 'tracker_anode_R2_ticks', 'tracker_anode_R3_ticks', 'tracker_anode_R4_ticks', 'tracker_bottom_cathode_R5_ticks', 'tracker_top_cathode_R6_ticks']


In [22]:
i = 0
data = {}
c_tdc2sec = 0.625E-08
t_tdc2sec = 1.25E-08
MAX_GG_TIMES = 10
for event in tree:
    run_id = event.run_id
    event_id = event.event_id
    # e_clock = event.event_clock                                         # int The clock of the events seconds
    # e_ticks = event.event_ticks                                         # Event times - TODO: not yet implemented
    n_calo_hits = int(event.nb_calo_hits)                               # int Number of calorimeter hits
    # calo_hit_id = list(event.calo_hit_id)                               # list(int) List of the calo hit IDs
    calo_om_side_id = list(event.calo_om_side_id)                       # list(int) List of OM sides (1: French, 0: Italian)
    calo_om_wall_id = list(event.calo_om_wall_id)                       # list(int) List of OM walls (0,1) see url:https://nemo.lpc-caen.in2p3.fr/wiki/NEMO/SuperNEMO/Calorimeter
    calo_om_column_id = list(event.calo_om_column_id)                   # list(int) List of OM columns
    calo_om_row_id = list(event.calo_om_row_id)                         # list(int) List of OM rows
    # calo_om_ref_id = list(event.calo_om_ref_id)                         # Don't know?
    # calo_clock = list(event.calo_clock)                                 # Don't know?
    calo_ticks = list(event.calo_ticks)                                 # list(int) Calo TDC (6.25ns)
    calo_lto = list(event.calo_lto)                                     # list(bool) Calo low threshold flag
    calo_ht = list(event.calo_ht)                                       # list(bool) Calo high threshold flag
    # calo_fcr = list(event.calo_fcr)                                   # list(int) Dont' know
    # calo_lt_trigger_counter = list(event.calo_lt_trigger_counter)       # Don't know
    # calo_lt_time_counter = list(event.calo_lt_time_counter)             # Don't know
    calo_fwmeas_baseline = list(event.calo_fwmeas_baseline)             # list(int) Calo baseline LSB: ADC unit/16
    calo_fwmeas_peak_amplitude = list(event.calo_fwmeas_peak_amplitude) # list(int) Calo amplitude LSB: ADC unit/8
    calo_fwmeas_peak_cell = list(event.calo_fwmeas_peak_cell)           # list(int) Calo peak cell TDC: 0-1023
    calo_fwmeas_charge = list(event.calo_fwmeas_charge)                 # list(int) Calo charge NO IDEA OF UNITS - do not trust this
    calo_fwmeas_rising_cell = list(event.calo_fwmeas_rising_cell)       # list(int) Calo rising cell (right edge) TDC unit/256
    calo_fwmeas_falling_cell = list(event.calo_fwmeas_falling_cell)     # list(int) Calo falling cell (left edge) TDC unit/256

    calo_waveform = [list(event.calo_waveform)[1024*i:1024*(i+1)] for i in range(n_calo_hits)]  # list(list(int)) Calo waveform

    nb_tracker_hits = event.nb_tracker_hits                                     # int Number of tracker cells that have at least one hit
    # tracker_hit_id = list(event.tracker_hit_id)                                 # list(int) TR hit ids
    tracker_cell_side_id = list(event.tracker_cell_side_id)                     # list(int) TR side (1: French, 0: Italian)
    tracker_cell_row_id = list(event.tracker_cell_row_id)                       # list(int) TR row (0-112)
    tracker_cell_layer_id = list(event.tracker_cell_layer_id)                   # list(int) TR layer (0-8)
    # tracker_clock = list(event.tracker_clock)                                   # list(int) Don't know
    tracker_anode_R0_ticks = [list(event.tracker_anode_R0_ticks)[i*MAX_GG_TIMES:MAX_GG_TIMES*(i+1)] for i in range(nb_tracker_hits)]                 # list(int) Anode time LSB: 12.5 ns
    tracker_anode_R1_ticks = [list(event.tracker_anode_R1_ticks)[i*MAX_GG_TIMES:MAX_GG_TIMES*(i+1)] for i in range(nb_tracker_hits)]                 # list(int) Anode: 1st low threshold LSB: 12.5 ns
    tracker_anode_R2_ticks = [list(event.tracker_anode_R2_ticks)[i*MAX_GG_TIMES:MAX_GG_TIMES*(i+1)] for i in range(nb_tracker_hits)]                # list(int) Anode: 2nd low threshold LSB: 12.5 ns
    tracker_anode_R3_ticks = [list(event.tracker_anode_R3_ticks)[i*MAX_GG_TIMES:MAX_GG_TIMES*(i+1)] for i in range(nb_tracker_hits)]                 # list(int) Anode: 1st high threshold LSB: 12.5 ns
    tracker_anode_R4_ticks = [list(event.tracker_anode_R4_ticks)[i*MAX_GG_TIMES:MAX_GG_TIMES*(i+1)] for i in range(nb_tracker_hits)]                 # list(int) Anode: 2st high threshold LSB: 12.5 ns
    tracker_bottom_cathode_R5_ticks = [list(event.tracker_bottom_cathode_R5_ticks)[i*MAX_GG_TIMES:MAX_GG_TIMES*(i+1)] for i in range(nb_tracker_hits)]   # list(int) Cathode bottom LSB: 12.5 ns
    tracker_top_cathode_R6_ticks = [list(event.tracker_top_cathode_R6_ticks)[i*MAX_GG_TIMES:MAX_GG_TIMES*(i+1)] for i in range(nb_tracker_hits)]         # list(int) Cathode top LSB: 12.5 ns

    if len(list(event.tracker_cell_side_id)) > 50:
        continue
    data[event_id] = {}
    for i_cell in range(nb_tracker_hits):
        cell = tracker_cell_side_id[i_cell] * 113 * 9 + tracker_cell_row_id[i_cell] * 9 + tracker_cell_layer_id[i_cell]
        data[event_id][cell] = {}
        rs = [tracker_anode_R0_ticks[i_cell],
              tracker_anode_R1_ticks[i_cell], tracker_anode_R2_ticks[i_cell],
              tracker_anode_R3_ticks[i_cell], tracker_anode_R4_ticks[i_cell], 
              tracker_bottom_cathode_R5_ticks[i_cell], tracker_top_cathode_R6_ticks[i_cell]]
        for index, r in enumerate(rs):
            data[event_id][cell][f'r{index}'] = r
    i +=1
    if i ==10:
        break

print(data[0].keys())
print(data[0])

dict_keys([1419, 1403, 1402, 1444, 1453, 1411, 1476, 1436, 1428, 1410, 1460, 1477, 1452, 1469, 1461])
{1419: {'r0': [870685441, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'r1': [870686998, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'r2': [870689370, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'r3': [870687093, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'r4': [870689466, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'r5': [870687006, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'r6': [-9223372036854775808, 0, 0, 0, 0, 0, 0, 0, 0, 0]}, 1403: {'r0': [870686233, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'r1': [870687362, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'r2': [870689481, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'r3': [870687434, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'r4': [870689554, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'r5': [870687357, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'r6': [870689478, 0, 0, 0, 0, 0, 0, 0, 0, 0]}, 1402: {'r0': [870685478, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'r1': [870686685, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'r2': [870688771, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'r3': [870686774, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'r4': [870688856, 0, 0, 0, 0, 0, 0,

In [51]:
Ntuple1 = uproot.open(input_file_name + ":RED")
Ntuple1.show()

name                 | typename                 | interpretation                
---------------------+--------------------------+-------------------------------
run_id               | int32_t                  | AsDtype('>i4')
event_id             | int32_t                  | AsDtype('>i4')
event_clock          | int16_t                  | AsDtype('>i2')
event_ticks          | int64_t                  | AsDtype('>i8')
nb_calo_hits         | uint32_t                 | AsDtype('>u4')
calo_hit_id          | int32_t[]                | AsJagged(AsDtype('>i4'))
calo_om_side_id      | int16_t[]                | AsJagged(AsDtype('>i2'))
calo_om_wall_id      | int16_t[]                | AsJagged(AsDtype('>i2'))
calo_om_column_id    | int16_t[]                | AsJagged(AsDtype('>i2'))
calo_om_row_id       | int16_t[]                | AsJagged(AsDtype('>i2'))
calo_om_ref_id       | int16_t[]                | AsJagged(AsDtype('>i2'))
calo_clock           | int16_t[]                | AsJagged(AsDt

In [113]:
ntp1df = Ntuple1.arrays(['event_id', 'tracker_anode_R0_ticks'], library='np')
ntp1df

{'event_id': array([   0,    1,    2, ..., 2199, 2200, 2201], dtype=int32),
 'tracker_anode_R0_ticks': array([array([[           870685441,                    0,                    0,
                                   0,                    0,                    0,
                                   0,                    0,                    0,
                                   0],
               [           870686233,                    0,                    0,
                                   0,                    0,                    0,
                                   0,                    0,                    0,
                                   0],
               [           870685478,                    0,                    0,
                                   0,                    0,                    0,
                                   0,                    0,                    0,
                                   0],
               [           870685532,      

In [93]:
calo_events = Ntuple1.arrays(["event_id", 'calo_om_side_id', 'calo_om_wall_id', 'calo_om_column_id', 'calo_om_row_id'], library='pd')
# calo_events = calo_events.set_index("event_id")
calo_events

event_id  calo_om_side_id  calo_om_wall_id  calo_om_column_id  \
entry subentry                                                                  
0     0                0                0               -1                  9   
      1                0                0               -1                  9   
      2                0                1               -1                  9   
      3                0                1               -1                  9   
1     0                1                0               -1                  9   
...                  ...              ...              ...                ...   
2199  3             2199                0               -1                  9   
2200  0             2200                1               -1                  8   
      1             2200                1               -1                  8   
2201  0             2201                1               -1                  9   
      1             2201                1               -1                  9   

                calo_om_row_id  
entry subentry                  
0     0                      2  
      1                      3  
      2                      6  
      3                      7  
1     0                      4  
...                        ...  
2199  3                      9  
2200  0                      4  
      1                      5  
2201  0                      4  
      1                      5  

[5741 rows x 5 columns]

In [104]:
print(calo_events["calo_om_side_id"][0])
print(calo_events["calo_om_side_id"][0][2])

subentry
0    0
1    0
2    1
3    1
Name: calo_om_side_id, dtype: int16
1
